In [1]:
import pandas as pd

In [25]:
df = pd.read_csv("geo-ide_dgv.csv", sep=";", encoding="latin-1")
df = df[df["dgv_match_id"].notnull()]
df[df.duplicated(subset="URL DCAT")]

,producteur,dgv_match_name,nb_MD,geoide_slug,dgv_match_id,URL DCAT
13,DDT 12 (Direction Départementale des Territoir...,DDT Aveyron,344,ddt 12 direction departementale territoires av...,58adbc9c88ee382998837258,http://www.geocatalogue.fr/www/dcat/8f8177d47c...
18,DDT 21 (Direction Départementale des Territoir...,DDT Côte-d'Or,1952,ddt 21 direction departementale territoires la...,58aec19a88ee385837ea9abe,http://www.geocatalogue.fr/www/dcat/8f8177d47c...


In [28]:
# FIXME
assert not df.duplicated(subset="URL DCAT").any()

AssertionError: 

In [74]:
import os
import requests

URL = "https://demo.data.gouv.fr/api/1/harvest"
auth = {
    "x-api-key": os.getenv("DATAGOUV_TOKEN")
}

def create_harvester(name, org_id, dcat_url):
    r = requests.post(f"{URL}/sources/", json={
       "active": True,
       "autoarchive": True,
       "backend": "dcat",
       "config":{},
       "name": f"[geocatalogue] {name}",
       "organization":{
          "id": org_id,
       },
       "url": dcat_url,
    }, headers=auth)
    if not r.ok:
        print(f"[ERROR] create harvester for {org_id}: {r.json()}")
        return
    data = r.json()
    harvester_id = data["id"]
    harvester_url = f"{URL}/source/{harvester_id}"
    r = requests.get(f"{harvester_url}/preview", headers=auth)
    if not r.ok:
        print(f"[ERROR] preview harvester for {org_id}: {r.text}")
        return
    data = r.json()
    if data["status"] == "failed":
        print(f"[ERROR] preview failed for org: {org_id}, harvester: {harvester_id}")
        return
    r = requests.post(f"{harvester_url}/validate", json={"state":"accepted", "comment": None}, headers=auth)
    if not r.ok:
        print(f"[ERROR] validate harvester for org: {org_id}, harvester: {harvester_id}, res: {r.text}")
        return
    print(f"[SUCCESS] org: {org_id}, harvester: {harvester_id}")

In [75]:
create_harvester("6", "534fff75a3a7292c64a77de4", "http://www.geocatalogue.fr/www/dcat/ec6481bf34fa43b527cb2424b2476bd3/catalog.rdf")

[SUCCESS] org: 534fff75a3a7292c64a77de4, harvester: 62bd53e268aba082731db0da
